In [135]:
import pandas as pd
import numpy as np
import requests
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split 
from sklearn import metrics 
from sklearn import tree
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

import sklearn as sk
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType 
from sklearn.datasets import load_iris

print('Packages Imported')

Packages Imported


In [136]:
# make call to API and put the results into a dataframe

url = 'https://opendata.utah.gov/resource/herb-zqda.json?$limit=50000&$$app_token=GunYY922rlATsFNuus6C4WyWE'
r = requests.get(url)
r = r.json()
df = pd.DataFrame(r)
df

,crash_id,crash_datetime,route,milepoint,lat_utm_y,long_utm_x,main_road_name,city,county_name,crash_severity_id,...,domestic_animal_related,overturn_rollover,commercial_motor_veh_involved,teenage_driver_involved,older_driver_involved,night_dark_condition,single_vehicle,distracted_driving,drowsy_driving,roadway_departure
0,11281387,2019-02-08T10:56:00.000,2258,4.56,4513124.085,422620.0512,900 WEST,SALT LAKE CITY,SALT LAKE,2,...,False,False,True,False,False,False,False,False,False,True
1,11282089,2019-12-24T12:48:00.000,2082,2.702,4483700.743,427458.6694,1215 E HIGHLAND DR,DRAPER,SALT LAKE,1,...,False,False,False,True,False,False,False,False,False,False
2,11282092,2019-12-28T20:42:00.000,2062,0.351,4486609.366,426779.0152,1022 E DRAPER PKY,DRAPER,SALT LAKE,1,...,False,False,False,True,False,True,False,False,False,False
3,11282098,2019-12-07T00:46:00.000,2048,0.438,4484350.913,426397.6578,13463 S FORT ST,DRAPER,SALT LAKE,3,...,False,True,False,False,False,True,True,False,False,False
4,11282101,2019-12-18T17:29:00.000,2068,0.796,4485217.604,427714.0392,13043 S 1300 E,DRAPER,SALT LAKE,1,...,False,False,False,True,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,11085099,2018-08-12T13:59:00.000,1082,9.983,4595883.291,407508.2587,3100 WEST,BRIGHAM CITY,BOX ELDER,1,...,False,False,False,False,False,False,False,True,False,False
49996,11085102,2018-08-11T22:33:00.000,2240,2.495,4502311.856,417417.3883,3605 W 4700 S,TAYLORSVILLE,SALT LAKE,1,...,False,False,False,False,False,True,False,False,False,False
49997,11085104,2018-08-04T13:45:00.000,56964,0.1,4632636.546,429578.3459,260 N,SMITHFIELD,CACHE,1,...,False,False,False,False,False,False,False,False,False,False
49998,11085107,2018-08-15T09:07:00.000,193,9.083,4551137.741,423097.5438,HWY 193,LAYTON,DAVIS,1,...,False,False,False,True,False,False,False,False,False,False


In [137]:



# Here we create a copy of the original dataframe. 
df2 = df

# Then we clean the data by removing any columns that have more than 30% missing values.
for i in df2:
    if (100 * df2[i].isnull().sum() / len(df)) > 30:
        # print(i)
        df2 = df2.drop(i, axis=1)


# Then print the percent of missing values in the dataframe
def missing_zero_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mz_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mz_table = mz_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mz_table['Data Type'] = df.dtypes
        mz_table = mz_table[
            mz_table.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        # print ("Your selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"      
        #     "There are " + str(mz_table.shape[0]) +
        #       " columns that have missing values.")
        return mz_table

print(missing_zero_values_table(df2))


 # Finally, we remove any rows with remaining missing values
df2 = df2.dropna()
# This prints the number of columns and rows that are still in the dataframe
print("Number of columns: " + str(len(df2.columns)))
print("Number of rows: " + str(len(df2)))




                   Missing Values  % of Total Values Data Type
city                          829                1.7    object
work_zone_related             391                0.8    object
milepoint                       3                0.0    object
route                           2                0.0    object
lat_utm_y                       2                0.0    object
long_utm_x                      2                0.0    object
main_road_name                  1                0.0    object
Number of columns: 29
Number of rows: 48776


In [138]:
# Now we separate the crash_datetime column into 4 separate columns
# of year, month, day, hour
df2['crash_datetime'] = pd.to_datetime(df2['crash_datetime'])
df2['year'], df2['month'], df2['day'], df2['hour'] = df2['crash_datetime'].dt.year, df2['crash_datetime'].dt.month, df2['crash_datetime'].dt.day, df2['crash_datetime'].dt.hour
df2 = df2.drop(columns=['crash_datetime'])

# To contiune our cleaning of the data, we bin certain values that represent less than 5%
# of the overall values. This function does that automatically based on the length of the dataframe 

def bin_groups(df, percent=.05):
  import pandas as pd
  for col in df:
    if not pd.api.types.is_numeric_dtype(df[col]):
      for group, count in df[col].value_counts().iteritems():
        if count / len(df) < percent:
          df.loc[df[col] == group, col] = 'Other'
  return df

# df2 = bin_groups(df2)


# After performing our bin function we noticed that the milepoint and the longitude, latitude, crash_id, and crash_severity_id were considered
# to be categorical when they should be considered #'s we we will change the data type and perform
# the bin function again

df2['crash_severity_id'] = df2['crash_severity_id'].astype(int)
df2['crash_id'] = df2['crash_id'].astype(int)
df2['milepoint'] = df2['milepoint'].astype(float)
df2['lat_utm_y'] = df2['lat_utm_y'].astype(float)
df2['long_utm_x'] = df2['long_utm_x'].astype(float)

df2 = bin_groups(df2)

df2


C:\Users\liber\AppData\Local\Temp\ipykernel_4304\3754401019.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['crash_datetime'] = pd.to_datetime(df2['crash_datetime'])
C:\Users\liber\AppData\Local\Temp\ipykernel_4304\3754401019.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['year'], df2['month'], df2['day'], df2['hour'] = df2['crash_datetime'].dt.year, df2['crash_datetime'].dt.month, df2['crash_datetime'].dt.day, df2['crash_datetime'].dt.hour
C:\Users\liber\AppData\Local\Temp\ipykernel_4304\3

KeyboardInterrupt: 

In [139]:
# Here we change the datatype of year, month, day and hour, and all
# true false fields to be of type object because we want to treat them 
# as categorical variables.
dfcopy = df2
dfcopy['year'] = dfcopy['year'].astype(object)
dfcopy['month'] = dfcopy['month'].astype(object)
dfcopy['day'] = dfcopy['day'].astype(object)
dfcopy['hour'] = dfcopy['hour'].astype(object)

# drop the id 
dfcopy2 = dfcopy.drop(columns=["crash_id"])

# dummy codes for all categorical variables
dfcopy2 = pd.get_dummies(dfcopy2, columns=[ 'pedestrian_involved',
       'bicyclist_involved', 'motorcycle_involved', 'improper_restraint',
       'unrestrained', 'dui', 'intersection_related', 'wild_animal_related',
       'domestic_animal_related', 'overturn_rollover',
       'commercial_motor_veh_involved', 'teenage_driver_involved',
       'older_driver_involved', 'night_dark_condition', 'single_vehicle',
       'distracted_driving', 'drowsy_driving', 'roadway_departure','route', 'main_road_name', 
       'milepoint', 'city', 'county_name', 'work_zone_related', 'year', 'month', 'day', 'hour'])



#print out the first 5 records of the dataframe
dfcopy2.columns

c:\Users\liber\Documents\byu-venv\lib\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


Index(['lat_utm_y', 'long_utm_x', 'crash_severity_id',
       'pedestrian_involved_False', 'pedestrian_involved_True',
       'bicyclist_involved_False', 'bicyclist_involved_True',
       'motorcycle_involved_False', 'motorcycle_involved_True',
       'improper_restraint_False',
       ...
       'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18', 'hour_19',
       'hour_20', 'hour_21', 'hour_22', 'hour_23'],
      dtype='object', length=36129)

In [140]:
# Feature selection for our model. We want to remove anything that is greater than .9

def fs_variance(df, label="", p=0.5):
          from sklearn.feature_selection import VarianceThreshold
          import pandas as pd
        
          if label != "":
            X = df.drop(columns=[label])
            
          sel = VarianceThreshold(threshold=(p * (1 - p)))
          sel.fit_transform(X)
        
          # Add the label back in after removing poor features
          return df[sel.get_feature_names_out()].join(df[label])

dfcopy2 = fs_variance(dfcopy2,label='crash_severity_id', p=0.9)

# Drop these columns for later on when we create the onnx file

dfcopy2 = dfcopy2.drop(columns=['teenage_driver_involved_False', 'older_driver_involved_False',
                          'night_dark_condition_False', 'single_vehicle_False', 'roadway_departure_False', 'milepoint_0.1', 
                          'intersection_related_False', 'year_2016', 'year_2017', 'year_2018'  ])

print(len(dfcopy2.columns))
print(dfcopy2.columns)




18
Index(['lat_utm_y', 'long_utm_x', 'intersection_related_True',
       'teenage_driver_involved_True', 'older_driver_involved_True',
       'night_dark_condition_True', 'single_vehicle_True',
       'roadway_departure_True', 'route_15', 'route_Other',
       'main_road_name_I-15', 'main_road_name_Other',
       'city_OUTSIDE CITY LIMITS', 'county_name_SALT LAKE', 'county_name_UTAH',
       'month_1', 'hour_17', 'crash_severity_id'],
      dtype='object')


In [141]:
# Now we set our X and Y so we can start training our Random Forest Classifier Classifier model
from sklearn.metrics import confusion_matrix

y = dfcopy2['crash_severity_id']
X = dfcopy2.drop(columns=['crash_severity_id'])


# This is where we split the data
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, random_state=12345, test_size=.3)


cv=KFold(n_splits=20,shuffle=True,random_state=12345)


# Now we need to hypertune the parameters. We will set out Random Forest Classifier with some basic settings
# Then we will search the grid to find the hyperparameters that give us the best score


param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

rf = se.RandomForestRegressor()

grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = cv, n_jobs = -1, verbose = 2)

grid_search.fit(Xtrain, ytrain)

print(grid_search.best_params_)
print(grid_search.best_estimator_)


1 0.7025448147797538
2 0.7025448147797538
3 0.7048879184622076
4 0.7051515045500756
5 0.704975706177484
6 0.703774751711166
7 0.7020759676713465
8 0.7002309165044334
9 0.6976826889965413


In [142]:
# Now we can run our Rnadom Forest Classifier model based on thsoe parameters
import sklearn.ensemble as se

# This sets up the model with the parameters
model_rf = se.RandomForestClassifier(n_estimators=10)

# This tells us the cross_validated R-Squared of our model
score = np.mean(cross_val_score(model_rf, Xtest, ytest, scoring='accuracy', cv=cv, n_jobs=-1))

score

0.6473752177195699

In [143]:
# Then we fit our model based on our trained values
randomforest = model_rf.fit(Xtrain, ytrain)

# Now we can predict based on our test data
ytestpred2 = randomforest.predict(Xtest)
# Now we output our predicted and actual values
output_df = pd.DataFrame({'Actual': ytest, 'Predicted': ytestpred2,})

output_df[0:5]


,Actual,Predicted
31989,1,1
45922,1,2
41004,1,1
23930,2,1
30565,1,1
28532,1,1
20094,1,1
26966,1,1
45992,1,1
35436,1,1


In [144]:
# Here we create our onnx file and save it on our local machine to later implement into our website
initial_type = [('float_input', FloatTensorType([None, 20]))]
onnx = convert_sklearn(model_rf, initial_types=initial_type)
with open("/Users/liber/OneDrive/Desktop/intex.onnx", "wb") as f:
    f.write(onnx.SerializeToString())

